In [1]:
import sys
sys.path.append('..')
import doctable
import dataclasses
import functools
import tempfile
import random
import copy

In [2]:
class ValueNotRetrievedEror(BaseException):
    pass

def get_getter_setter(property_name: str):
    class TmpGetterSetter:
        @property
        def a(self):
            if getattr(self, property_name) is dataclasses._MISSING_TYPE:
                raise ValueNotRetrievedEror(f'The "{property_name[1:]}" property '
                    'was never retrieved from the database. This might appear if '
                    'you specified columns in your SELECT statement.')
            return getattr(self, property_name)
        
        @a.setter
        def a(self, val):
            setattr(self, property_name, val)
    return TmpGetterSetter.a


def schema_properties(_Cls=None, *, require_slots=True, **dataclass_kwargs):

    # this is the actual decorator
    def decorator_schema(Cls):
        # creates constructor/other methods using dataclasses
        Cls = dataclasses.dataclass(Cls, **dataclass_kwargs)
        
        slot_var_names = list()
        for field in dataclasses.fields(Cls):
            property_name = f'_{field.name}'
            
            # dataclasses don't actually create the property unless the default
            # value was a constant, so we just want to replicate that behavior
            # with EmptyValue as the object. Normally it would do that in the 
            # constructor, but we want to create it ahead of time.
            if hasattr(Cls, field.name):
                setattr(Cls, property_name, getattr(Cls, field.name))
            else:
                setattr(Cls, property_name, doctable.MISSING_VALUE)
            
            # used a function to generate a class and return the property
            # to solve issue with class definitions in loops
            setattr(Cls, field.name, get_getter_setter(property_name))
        
        # add slots
        if require_slots and not hasattr(Cls, '__slots__'):
            raise doctable.SlotsRequiredError()
        
        @functools.wraps(Cls, updated=[])
        class NewClass(doctable.DocTableSchema, Cls):
            __slots__ = [f.name for f in dataclasses.fields(Cls)]
        
        return NewClass

    if _Cls is None:
        return decorator_schema
    else:
        return decorator_schema(_Cls)

In [10]:

@schema_properties
class MyObj:
    __slots__ = []
    id: int = doctable.Col()
    name: str = doctable.Col()
    extra1: str = doctable.Col()
    extra2: str = doctable.Col()
    extra3: str = doctable.Col()
    extra4: str = doctable.Col()
    extra5: str = doctable.Col()

objects = [MyObj(id=i, name=f'user_{i}') for i in range(1000)]
#print(objects[0].__doctable_colnames)
object_dicts = [o._doctable_as_dict() for o in objects]
object_dicts[0]

objects[0].__slots__

['id', 'name', 'extra1', 'extra2', 'extra3', 'extra4', 'extra5']

In [4]:
# benchmarking some basic tasks that happen during insertion
%timeit dataclasses.is_dataclass(MyObj) # time it takes to check if myobj is a dataclass
%timeit objects[0]._doctable_as_dict() # time it takes to convert myobj to dict

211 ns ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


KeyboardInterrupt: 

In [ ]:
def get_newtab(tempfolder: tempfile.TemporaryDirectory):
    target = f'{tempfolder.name}/{random.randrange(9999)}.db'
    tab = doctable.DocTable(target=target, schema=MyObj, new_db=True)
    tab.delete()
    assert(tab.count() == 0)
    return tab

tmpf = tempfile.TemporaryDirectory('mytmp')
tab = get_newtab(tmpf)

In [ ]:
tab = get_newtab(tmpf)
%timeit tab.insert(objects)
tab = get_newtab(tmpf)
%timeit tab.insert(object_dicts)

31.4 ms ± 354 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.1 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
import typing
@dataclasses.dataclass
class MyFunObj:
    id: int
    name: str = dataclasses._MISSING_TYPE
    friends: typing.List[str] = dataclasses.field(default_factory=list)

class ValueNotRetrievedEror(BaseException):
    pass

def get_tmp_getter_setter(property_name: str):
    class TmpGetterSetter:
        @property
        def a(self):
            if getattr(self, property_name) is dataclasses._MISSING_TYPE:
                raise ValueNotRetrievedEror(f'The "{property_name[1:]}" property '
                                'was never retrieved from the database.')
            return getattr(self, property_name)
        
        @a.setter
        def a(self, val):
            setattr(self, property_name, val)
    return TmpGetterSetter.a


for field in dataclasses.fields(MyFunObj):
    colname = field.name
    
    # this is the name of the getter/setter properties
    property_name = f'_{colname}'
    
    # dataclasses don't actually create the property unless the default
    # value was a constant, so we just want to replicate that behavior
    # with EmptyValue as the object. Normally it would do that in the 
    # constructor, but we want to create it ahead of time.
    if hasattr(MyFunObj, field.name):
        setattr(MyFunObj, property_name, getattr(MyFunObj, field.name))
    else:
        setattr(MyFunObj, property_name, dataclasses._MISSING_TYPE)
    
    # used a function to generate a class and return the property
    # to solve issue with class definitions in loops
    setattr(MyFunObj, field.name, get_tmp_getter_setter(property_name))

print(dir(MyFunObj))
o = MyFunObj(1)
o.id = 100
o._id, o

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_friends', '_id', '_name', 'friends', 'id', 'name']


ValueNotRetrievedEror: The "name" property was never retrieved from the database.